# Getting Started with monteplan

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/engineerinvestor/monteplan/blob/main/notebooks/01_getting_started.ipynb)

This notebook introduces Monte Carlo retirement simulation with monteplan. You'll learn to:

1. Run a simulation with default parameters
2. Interpret success probability and terminal wealth
3. Visualize wealth trajectories with a fan chart
4. Customize your plan and market assumptions
5. Export and import configurations

In [ ]:
# Uncomment and run in Google Colab:
# !pip install -q "monteplan @ git+https://github.com/engineerinvestor/monteplan.git"

## What is Monte Carlo Simulation?

Monte Carlo simulation models uncertainty by running thousands of random scenarios. For retirement planning, each scenario ("path") generates a different sequence of market returns, inflation rates, and portfolio outcomes. By examining thousands of paths, we can estimate the probability that a financial plan will succeed (portfolio lasts until the end of the planning horizon).

In [ ]:
from monteplan.config.defaults import (
    default_plan, default_market, default_policies, default_sim_config
)
from monteplan.core.engine import simulate

## The Default Plan

Let's look at what the default plan assumes:

In [ ]:
plan = default_plan()
print(f"Current age: {plan.current_age}")
print(f"Retirement age: {plan.retirement_age}")
print(f"Planning horizon: to age {plan.end_age}")
print(f"Monthly spending in retirement: ${plan.monthly_spending:,.0f}")
print(f"\nAccounts:")
for acct in plan.accounts:
    print(f"  {acct.account_type}: ${acct.balance:,.0f} balance, ${acct.annual_contribution:,.0f}/yr contribution")

The default plan models a 30-year-old who:
- Retires at 65
- Plans to age 95 (65-year total horizon)
- Has 3 investment accounts (taxable, traditional 401k, Roth)
- Spends $5,000/month in retirement (in today's dollars)

## Run the Simulation

In [ ]:
market = default_market()
policies = default_policies()
sim_config = default_sim_config()

result = simulate(plan, market, policies, sim_config)

print(f"Success probability: {result.success_probability:.1%}")
print(f"\nTerminal wealth percentiles (at age {plan.end_age}):")
for key, value in sorted(result.terminal_wealth_percentiles.items()):
    print(f"  {key}: ${value:,.0f}")

## Interpreting the Results

**Success probability** tells you what fraction of the 5,000 simulated futures resulted in the portfolio lasting through age 95. With the default 6-asset globally diversified portfolio, the plan succeeds in roughly 76% of scenarios.

**Terminal wealth percentiles** show the distribution of final portfolio values:
- **p5**: In the worst 5% of scenarios, the portfolio is worth this much (or less)
- **p50**: The median outcome
- **p95**: In the best 5% of scenarios

## Fan Chart

A fan chart shows the range of possible wealth trajectories over time:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

ts = result.wealth_time_series
ages = np.linspace(plan.current_age, plan.end_age, len(ts["p50"]))

fig, ax = plt.subplots(figsize=(10, 5))
ax.fill_between(ages, ts["p5"], ts["p95"], alpha=0.15, color="steelblue", label="P5-P95")
ax.fill_between(ages, ts["p25"], ts["p75"], alpha=0.3, color="steelblue", label="P25-P75")
ax.plot(ages, ts["p50"], color="steelblue", linewidth=2, label="Median (P50)")
ax.axvline(plan.retirement_age, color="gray", linestyle="--", alpha=0.5, label="Retirement")
ax.set_xlabel("Age")
ax.set_ylabel("Portfolio Value ($)")
ax.set_title("Wealth Fan Chart")
ax.legend()
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"${x/1e6:.1f}M" if x >= 1e6 else f"${x/1e3:.0f}K"))
plt.tight_layout()
plt.show()

The fan chart shows:
- **Dark band** (P25-P75): The middle 50% of outcomes
- **Light band** (P5-P95): The middle 90% of outcomes
- **Solid line**: Median trajectory
- **Dashed line**: Retirement age

The portfolio grows during the accumulation phase (contributions + returns), then draws down in retirement.

## Customize Your Plan

Let's create a custom plan and compare it to the default:

In [ ]:
from monteplan.config.schema import PlanConfig, AccountConfig

# Higher savings, lower spending plan
custom_plan = PlanConfig(
    current_age=30,
    retirement_age=60,  # Retire 5 years earlier
    end_age=95,
    accounts=[
        AccountConfig(account_type="taxable", balance=50_000, annual_contribution=12_000),
        AccountConfig(account_type="traditional", balance=100_000, annual_contribution=23_000),
        AccountConfig(account_type="roth", balance=30_000, annual_contribution=7_000),
    ],
    monthly_income=10_000,
    monthly_spending=4_000,  # Lower spending
)

custom_result = simulate(custom_plan, market, policies, sim_config)
print(f"Default plan success: {result.success_probability:.1%}")
print(f"Custom plan success:  {custom_result.success_probability:.1%}")

In [ ]:
# Side-by-side fan charts
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

for ax, res, p, title in [
    (ax1, result, plan, f"Default Plan ({result.success_probability:.1%})"),
    (ax2, custom_result, custom_plan, f"Custom Plan ({custom_result.success_probability:.1%})"),
]:
    ts = res.wealth_time_series
    ages = np.linspace(p.current_age, p.end_age, len(ts["p50"]))
    ax.fill_between(ages, ts["p5"], ts["p95"], alpha=0.15, color="steelblue")
    ax.fill_between(ages, ts["p25"], ts["p75"], alpha=0.3, color="steelblue")
    ax.plot(ages, ts["p50"], color="steelblue", linewidth=2)
    ax.axvline(p.retirement_age, color="gray", linestyle="--", alpha=0.5)
    ax.set_xlabel("Age")
    ax.set_title(title)
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"${x/1e6:.1f}M" if x >= 1e6 else f"${x/1e3:.0f}K"))

ax1.set_ylabel("Portfolio Value ($)")
plt.tight_layout()
plt.show()

## Change Market Assumptions

The default market uses 6 globally diversified assets (US/Ex-US Dev/EM for both stocks and bonds). You can switch to a simpler 2-asset US-only market, or customize returns and volatilities:

In [ ]:
from monteplan.config.schema import MarketAssumptions, AssetClass
from monteplan.config.defaults import us_only_market

# The default market is a 6-asset globally diversified portfolio.
# You can also use a simple 2-asset US-only market:
simple_market = us_only_market()
print(f"US-only market: {len(simple_market.assets)} assets")
for a in simple_market.assets:
    print(f"  {a.name}: {a.weight:.0%}")

# Or build a fully custom market with optimistic assumptions:
optimistic_market = default_market()  # Start with 6-asset global defaults
optimistic_market = optimistic_market.model_copy(update={
    "expected_annual_returns": [0.09, 0.085, 0.095, 0.06, 0.055, 0.07],  # Higher returns
    "inflation_mean": 0.025,  # Lower inflation
})

optimistic_result = simulate(plan, optimistic_market, policies, sim_config)
print(f"\nDefault market: {result.success_probability:.1%}")
print(f"Optimistic market: {optimistic_result.success_probability:.1%}")

## Export and Import Configurations

Save your configuration as JSON for reproducibility:

In [ ]:
from monteplan.io.serialize import dump_config, load_config

# Export to JSON
json_str = dump_config(plan, market, policies, sim_config)
print(json_str[:300] + "...")

# Round-trip: load it back
loaded_plan, loaded_market, loaded_policies, loaded_sim = load_config(json_str)
print(f"\nLoaded plan age range: {loaded_plan.current_age} to {loaded_plan.end_age}")
print(f"Config hash: {result.config_hash[:16]}...")

## Next Steps

- **[02 - Spending Policies](02_spending_policies.ipynb)**: Compare the 5 available spending strategies
- **[03 - Advanced Analysis](03_advanced_analysis.ipynb)**: Return models, stress tests, and sensitivity analysis
- **[04 - FIRE Case Study](04_case_study_fire.ipynb)**: Early retirement at 45
- **[05 - Retiree Case Study](05_case_study_retiree.ipynb)**: Conservative retiree with Social Security
- **[Documentation](https://engineerinvestor.github.io/monteplan/)**: Full documentation site
- **Streamlit App**: `pip install monteplan[app] && streamlit run app/Home.py`